In [1]:
from unitree_robot.train.environments import MujocoEnv, Go2Env
from unitree_robot.train.training import Trainer
from unitree_robot.train.experiments import Experiment

from gymnasium.spaces import Space
import numpy as np
import quaternion

import torch as T

In [2]:
MJCF_PATH = "./external/mujoco_mangerie/unitree_go2/scene-flat-ground.xml"
SEED = 0

np.random.seed(seed=SEED)

env = Go2Env(
    model_path=MJCF_PATH,
    sim_frames_per_step=5,
    seed=SEED,
    experiment=Experiment()
)

obs = env.reset(seed=SEED)

env.model.camera("main").pos = np.array([1, 1, 3])

trainer = Trainer(
    env=env,
    device="cpu",
    network_hidden_size=16,
)

Trainer: device set to cpu !


In [3]:
trainer.train()

TypeError: Experiment.calculate_loss() takes 1 positional argument but 2 were given

In [7]:
try:
    i = 0
    while True:
        
        action = env.action_space.sample()
        env.do_simulation(ctrl=action, n_frames=5)

        env.render(cam_offset=np.array([1,1,5]))

        # print(env.data.qfrc_actuator)
        
        i += 1
        
except Exception as e:
    raise e
finally:
    env.close()

[  0.           0.           0.           0.           0.
   0.          21.51800728   1.08320487 -15.23073578 -20.95428658
   7.59899855  -7.06027222  -2.25599885   9.46981525  18.9301548
  -9.26308441 -10.14897442 -30.14691353]
[-117.39896548  -49.82367398  127.53397816    2.4460084    14.14363298
  -19.53449438   -0.2           0.2           0.2          19.06774943
   -0.2           0.2           0.2          -0.13471311   -0.2
    0.2           0.2           0.2       ]


In [9]:
from unitree_robot.train.environments import lookat


target = env.data.cam("main").xpos + np.array([0, 0, -1])
target

# lookat(env.data.cam("main").xpos, target)


In [2]:
# env.action_space

# len(env.data.qpos)

# env.data.("imu_quat")


# q_rotate = quaternion.as_quat_array(env.data.sensor("imu_quat").data)
# quaternion.rotate_vectors(q_rotate, np.array([0,0,1]))
# (np.array([1, 0, 0]))




# env.get_sensor_state()

# env.data.xquat.shape